In [2]:
from langgraph.graph import StateGraph , START , END
from langchain_ollama import ChatOllama
from typing import Annotated , TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv

In [3]:
load_dotenv()

llm = ChatOllama(model='gemma:2b')

In [4]:
class JokeState(TypedDict):
    
    topic : str
    joke : str
    explanation : str

In [5]:
def generate_joke(state : JokeState) -> JokeState:
    
    prompt = f"Generate a joke about {state['topic']}"
    
    response = llm.invoke(prompt).content
    
    return {'joke' : response}

In [6]:
def generate_explanation(state : JokeState) -> JokeState:
    
    prompt = f"Explain the joke '{state['joke']}'"

    response = llm.invoke(prompt).content

    return {'explanation' : response}

In [10]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke' , generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START , 'generate_joke')
graph.add_edge('generate_joke' , 'generate_explanation')
graph.add_edge('generate_explanation' , END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)